In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

from glmext.glm import NegativeBinomialRegressor


def generate_data(n_samples=1000, n_features=5, k=0.5, random_state=42):
    """
    Generate synthetic data for Negative Binomial regression.
    k: Dispersion parameter (1/theta). Var = mu + k * mu^2
    """
    rng = np.random.RandomState(random_state)
    X = rng.randn(n_samples, n_features)
    # True coefficients
    coef = rng.randn(n_features) * 0.5
    intercept = 1.5
    # Linear predictor
    linear_pred = np.dot(X, coef) + intercept
    mu = np.exp(linear_pred)
    # Generate y using Negative Binomial distribution
    # scipy/numpy uses n (successes) and p (probability) parameterization
    # mean = n * (1-p) / p
    # var = mean / p
    # Relation to k (alpha in statsmodels): n = 1/k, p = 1 / (1 + k*mu)
    if k > 0:
        n_nb = 1 / k
        p_nb = 1 / (1 + k * mu)
        y = rng.negative_binomial(n_nb, p_nb)
    else:
        y = rng.poisson(mu)
    return X, y, coef, intercept


def check_models():
    # 1. Setup Data
    n_samples = 2000
    n_features = 3
    k_true = 3 # Dispersion
    X, y, true_coef, true_intercept = generate_data(n_samples, n_features, k=k_true)
    print(f"--- Data Generation ---")
    print(f"True Dispersion (k): {k_true}")
    print(f"Mean of y: {np.mean(y):.4f}, Var of y: {np.var(y):.4f}")
    print(f"Ratio Var/Mean: {np.var(y)/np.mean(y):.4f} (Expect > 1)\n")

    # 2. Statsmodels Implementation
    # Statsmodels parameterizes NegativeBinomial with 'alpha' which corresponds to our 'k'
    # We must assume 'alpha' is fixed to the known truth for a direct comparison of solvers,
    # or we can let statsmodels estimate it.
    # Here, we fit with the KNOWN k (alpha) to check if coefficients match.
    # Add intercept for statsmodels
    X_sm = sm.add_constant(X)
    # statsmodels NegativeBinomial family
    # loglike_method='nb2' corresponds to Var = mu + alpha * mu^2
    sm_model = sm.GLM(
        y,
        X_sm,
        family=sm.families.NegativeBinomial(alpha=k_true)
    ).fit()

    print("--- Statsmodels (GLM) ---")
    print(sm_model.summary().tables[1])
    sm_coef = sm_model.params[1:]
    sm_intercept = sm_model.params[0]
    # 3. Custom Sklearn Implementation
    # We set alpha=0 to remove regularization (matching statsmodels default)
    # We fix k to the same value used in statsmodels
    custom_model = NegativeBinomialRegressor(
        k=k_true,
        alpha=0.0, # No L2 regularization
        fit_intercept=False,
        solver='lbfgs',
        max_iter=1000,
        tol=1e-6
    )
    custom_model.fit(
        X_sm, y
    )
    print("\n--- Custom NegativeBinomialRegressor ---")
    print(f"Intercept: {custom_model.coef_[0]:.4f}")
    print(f"Coefficients: {custom_model.coef_[1:]}")
    # 4. Comparison
    print("\n--- Comparison (Absolute Difference) ---")
    diff_intercept = abs(sm_intercept - custom_model.coef_[0])
    diff_coef = np.abs(sm_coef - custom_model.coef_[1:])
    print(f"Intercept Diff: {diff_intercept:.6f}")
    print(f"Coef Max Diff : {np.max(diff_coef):.6f}")
    if diff_intercept < 1e-3 and np.max(diff_coef) < 1e-3:
        print("SUCCESS: Models match closely!")
    else:
        print("Models diverge. Check regularization or tolerance.")

    # 5. Prediction Check
    y_pred_sm = sm_model.predict(X_sm)
    y_pred_custom = custom_model.predict(X_sm)
    print(f"\nMean Prediction Diff: {np.mean(np.abs(y_pred_sm - y_pred_custom)):.6f}")


check_models()

--- Data Generation ---
True Dispersion (k): 3
Mean of y: 5.7900, Var of y: 162.2509
Ratio Var/Mean: 28.0226 (Expect > 1)

--- Statsmodels (GLM) ---
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5332      0.040     37.883      0.000       1.454       1.613
x1            -0.5092      0.041    -12.496      0.000      -0.589      -0.429
x2            -0.2434      0.041     -6.003      0.000      -0.323      -0.164
x3            -0.4067      0.041     -9.935      0.000      -0.487      -0.326

--- Custom NegativeBinomialRegressor ---
Intercept: 1.5332
Coefficients: [-0.50915684 -0.24337279 -0.40665888]

--- Comparison (Absolute Difference) ---
Intercept Diff: 0.000000
Coef Max Diff : 0.000000
SUCCESS: Models match closely!

Mean Prediction Diff: 0.000001


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

from glmext.glm import BinomialRegressor


def generate_data(n_samples=1000, n_features=5, k=0.5, random_state=42):
    """
    Generate synthetic data for Binomial regression.
    """
    rng = np.random.RandomState(random_state)
    X = rng.randn(n_samples, n_features)
    # True coefficients
    coef = rng.uniform(-1, 1, size=n_features)
    intercept = 0.5
    # Linear predictor
    linear_pred = np.dot(X, coef) + intercept
    p = 1 / (1 + np.exp(-linear_pred))
    y = rng.binomial(n=1, p=p)
    return X, y, coef, intercept


def check_models():
    # 1. Setup Data
    n_samples = 2000
    n_features = 3
    k_true = 3 # Dispersion
    X, y, true_coef, true_intercept = generate_data(n_samples, n_features, k=k_true)
    print(f"--- Data Generation ---")
    print(f"True Dispersion (k): {k_true}")
    print(f"Mean of y: {np.mean(y):.4f}, Var of y: {np.var(y):.4f}")
    print(f"Ratio Var/Mean: {np.var(y)/np.mean(y):.4f} (Expect > 1)\n")

    # 2. Statsmodels Implementation
    # Add intercept for statsmodels
    X_sm = sm.add_constant(X)
    # statsmodels Binomial family
    # loglike_method='nb2' corresponds to Var = mu + alpha * mu^2
    sm_model = sm.GLM(
        y,
        X_sm,
        family=sm.families.Binomial()
    ).fit()

    print("--- Statsmodels (GLM) ---")
    print(sm_model.summary().tables[1])
    sm_coef = sm_model.params[1:]
    sm_intercept = sm_model.params[0]
    # 3. Custom Sklearn Implementation
    # We set alpha=0 to remove regularization (matching statsmodels default)
    # We fix k to the same value used in statsmodels
    custom_model = BinomialRegressor(
        alpha=0.0, # No L2 regularization
        fit_intercept=False,
        solver='lbfgs',
        max_iter=1000,
        tol=1e-6
    )
    custom_model.fit(
        X_sm, y
    )
    print("\n--- Custom BinomialRegressor ---")
    print(f"Intercept: {custom_model.coef_[0]:.4f}")
    print(f"Coefficients: {custom_model.coef_[1:]}")
    # 4. Comparison
    print("\n--- Comparison (Absolute Difference) ---")
    diff_intercept = abs(sm_intercept - custom_model.coef_[0])
    diff_coef = np.abs(sm_coef - custom_model.coef_[1:])
    print(f"Intercept Diff: {diff_intercept:.6f}")
    print(f"Coef Max Diff : {np.max(diff_coef):.6f}")
    if diff_intercept < 1e-3 and np.max(diff_coef) < 1e-3:
        print("SUCCESS: Models match closely!")
    else:
        print("Models diverge. Check regularization or tolerance.")

    # 5. Prediction Check
    y_pred_sm = sm_model.predict(X_sm)
    y_pred_custom = custom_model.predict(X_sm)
    print(f"\nMean Prediction Diff: {np.mean(np.abs(y_pred_sm - y_pred_custom)):.6f}")


check_models()

--- Data Generation ---
True Dispersion (k): 3
Mean of y: 0.6195, Var of y: 0.2357
Ratio Var/Mean: 0.3805 (Expect > 1)

--- Statsmodels (GLM) ---
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5373      0.049     10.890      0.000       0.441       0.634
x1            -0.3606      0.050     -7.263      0.000      -0.458      -0.263
x2            -0.5790      0.052    -11.050      0.000      -0.682      -0.476
x3            -0.3579      0.051     -7.046      0.000      -0.458      -0.258

--- Custom BinomialRegressor ---
Intercept: 0.5373
Coefficients: [-0.36058157 -0.57901396 -0.3579421 ]

--- Comparison (Absolute Difference) ---
Intercept Diff: 0.000000
Coef Max Diff : 0.000000
SUCCESS: Models match closely!

Mean Prediction Diff: 0.000000
